# Imports and configuration.

In [1]:
%matplotlib inline

In [9]:
import lda
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text


import functions as f

In [3]:
matplotlib.style.use('ggplot')

# Load data

In [6]:
con = psycopg2.connect("host='localhost' dbname='explore' user='explore' password='Ln2bOYAVCG6utNUSaSZaIVMH'")
cursor = con.cursor()

cursor.execute('select category, count(*) from recipes_recipe group by category')

df = pd.read_sql_query('select * from recipes_recipe', con=con)
print('Loaded %s records' % df.shape[0])


Loaded 4506 records


# Run model

In [11]:
vectorizer = CountVectorizer(stop_words='english',
                                 ngram_range=(1, 1),
                                 token_pattern='[A-Za-z]+')
features = vectorizer.fit_transform(df.ingredient_txt)
## features is a document x term matrix.

wc = f.feature_counts(vectorizer, features)
m = lda.LDA(n_topics=40, random_state=0, n_iter=100)
m.fit(features)


<lda.lda.LDA instance at 0x7f17f51b6560>

# Evaluating the model.
## Convergence

In [31]:
plt.plot(m.loglikelihoods_, '-')
plt.title('Loglikelihood')


# Assessing topics

In [128]:
## Extracting topic data.
reload(f)

## most probable words by topic.
w = f.most_probable_words(m, vectorizer.get_feature_names(), 10)

## most probable documents by topic.
np.apply_along_axis(lambda i: df.iloc[i]['title'], 1, doc_ids)

doc_ids = np.argsort(m.doc_topic_, axis=0)[-4:-1,:].T

for t in range(m.n_topics):
    print('topic: %s' % t)
    print(', '.join(w[w['label']==t]['word']))
    print('documents:')
    print(', '.join(df.iloc[doc_ids[t,:]]['title'])
    print('-'*70)




topic: 0
lime, tablespoon, plus, sugar, lemon, tablespoons, cups, juice, cup
----------------------------------------------------------------------
topic: 1
pinch, sea, water, lb, cup, ml, tsp, oz, tbsp
----------------------------------------------------------------------
topic: 2
fresh, garlic, medium, celery, white, broth, cup, chicken, cups
----------------------------------------------------------------------
topic: 3
powder, butter, cups, purpose, large, sugar, baking, flour, cup
----------------------------------------------------------------------
topic: 4
french, breakfast, free, taste, fat, needed, goya, s, milk
----------------------------------------------------------------------
topic: 5
salt, cups, heavy, egg, vanilla, large, teaspoon, cream, sugar
----------------------------------------------------------------------
topic: 6
ice, syrup, lime, fresh, garnish, lemon, cups, juice, ounce
----------------------------------------------------------------------
topic: 7
sliced,